## 차트 크롤링

In [8]:
%pip install --upgrade --force-reinstall numpy

     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------------ ------------------------- 20.5/60.8 kB 330.3 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 648.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   --- ------------------------------------ 1.0/12.9 MB 22.0 MB/s eta 0:00:01
   ---------- ----------------------------- 3.3/12.9 MB 35.6 MB/s eta 0:00:01
   ----------------- ---------------------- 5.6/12.9 MB 40.1 MB/s eta 0:00:01
   ------------------------- -------------- 8.4/12.9 MB 41.1 MB/s eta 0:00:01
   ------------------------------- -------- 10.3/12.9 MB 43.5 MB/s eta 0:00:01
   ------------------------------------ --- 11.7/12.9 MB 40.9 MB/s eta 0:00:01
   ---------------------------------------  12.9/12.9 MB 40.9 MB/s eta 0:00:01
   ---------------------------------------- 12.9/12.9 MB 34.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2

  You can safely remove it manually.
  You can safely remove it manually.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\bsj03\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
from ssl import Options
from selenium import webdriver as wd
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup
import pandas as pd
import re


chrome_options = Options()  # ChromeOptions로 변경
# chrome_options.add_argument('--headless')  # Headless 모드
# chrome_options.add_argument('--disable-gpu')  # GPU 비활성화 (Windows에서 필요)
# chrome_options.add_argument('--no-sandbox')  # 샌드박스 비활성화 (리눅스에서 필요)

#연대
period = 1
#연도
year = 1 
# 월
month = 1
# 장르 ( 4 ~ 9)
genre = 4
chart_data = []

for year in range(2,7):
    for month in range(1,13):  # 1부터 12까지 월을 반복
        try:
            # ChromeDriver 경로 설정
            # path = 'c:/Users/PRO/.wdm/drivers/chromedriver/win64/131.0.6778.204/chromedriver-win32/chromedriver.exe'
            # service = Service(path)

            # ChromeDriver 자동 설치 및 실행
            service = Service(ChromeDriverManager().install())
            driver = wd.Chrome(service=service, options=chrome_options)

            # WebDriver 초기화
            # driver = wd.Chrome(service=service, options=chrome_options)
            driver.maximize_window()

            # URL 접속
            url = 'https://www.melon.com/chart/index.htm'
            driver.get(url)
            time.sleep(0.5)

            # 차트 파인더 클릭
            driver.find_element(By.XPATH, '//*[@id="gnb_menu"]/ul[1]/li[1]/div/div/button/span').click()
            time.sleep(0.5)

            # 월간 차트 클릭
            # //*[@id="d_chart_search"]/div/h4[2]/a
            driver.find_element(By.XPATH, '//*[@id="d_chart_search"]/div/h4[2]/a').click()
            time.sleep(1)

            # 연대 선택
            # 2010년대 //*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li[2]/span/label
            driver.find_element(By.XPATH, f'//*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li[{period}]/span/label').click()
            time.sleep(1)

            # 연도 선택 (연도 목록 추출)
            # year_elements = driver.find_elements(By.XPATH, '//*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li/span/label')
            # years = [year.text.strip() for year in year_elements]
            # print("Available years:", years)

            # 원하는 연도 선택 (예시로 첫 번째 연도를 선택)
            driver.find_element(By.XPATH, f'//*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[{year}]/span/label').click()
            time.sleep(1)

            # 월 선택
            # //*[@id="d_chart_search"]/div/div/div[3]/div[1]/ul/li[1]/span/label
            driver.find_element(By.XPATH, f'//*[@id="d_chart_search"]/div/div/div[3]/div[1]/ul/li[{month}]/span/label').click()
            time.sleep(1)
            
            for genre in range(4,10):
                # 장르 선택
                driver.find_element(By.XPATH, f'//*[@id="d_chart_search"]/div/div/div[5]/div[1]/ul/li[{genre}]/span/label').click()
                time.sleep(1)

                # 검색 버튼 클릭
                driver.find_element(By.XPATH, '//*[@id="d_srch_form"]/div[2]/button/span/span').click()
                time.sleep(1)

                # 데이터 크롤링 수행
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')

                # lst50과 lst100 두 클래스의 곡을 모두 가져옴
                songs = soup.find_all('tr', {'class': 'lst50'}) + soup.find_all('tr', {'class': 'lst100'})

                # 곡별 데이터 추출
                for idx, song in enumerate(songs, start=1):
                    title = song.find('strong')
                    title = title.text.strip() if title else 'Unknown Title'

                    artist = song.find('div', {'class': 'ellipsis rank02'})
                    artist = artist.text.strip() if artist else 'Unknown Artist'

                    song_link = song.find('a', {'class': 'btn_icon_detail'})  # 해당 버튼 찾기
                    songID = re.search(r"goSongDetail\('(\d+)'\)", song_link['href']).group(1) if song_link else 'Unknown ID'

                    chart_data.append({
                        'Title': title,
                        'Artist': artist,
                        'SongID': songID,
                    })

            driver.quit()

        except Exception as e:
            print(f"Error during month {month}: {e}")
            break

df = pd.DataFrame(chart_data)

df.to_csv('melon_crawling_2024.csv', index=False, encoding='utf-8-sig')
print(df.head())


KeyboardInterrupt: 

In [11]:
chart_data

[{'Rank': '1', 'Title': '비의 랩소디', 'Artist': '임재현임재현', 'SongID': '37023625'},
 {'Rank': '2', 'Title': '헤어지자 말해요', 'Artist': '박재정박재정', 'SongID': '36382580'},
 {'Rank': '3', 'Title': '인사', 'Artist': '범진범진', 'SongID': '34451383'},
 {'Rank': '4',
  'Title': '그대만 있다면 (여름날 우리 X 너드커넥션 (Nerd Connection))',
  'Artist': '너드커넥션 (Nerd Connection)너드커넥션 (Nerd Connection)',
  'SongID': '36699489'},
 {'Rank': '5', 'Title': '첫 눈', 'Artist': 'EXOEXO', 'SongID': '4352438'},
 {'Rank': '6', 'Title': '너의 모든 순간', 'Artist': '성시경성시경', 'SongID': '4446485'},
 {'Rank': '7',
  'Title': '사막에서 꽃을 피우듯',
  'Artist': '우디 (Woody)우디 (Woody)',
  'SongID': '36616378'},
 {'Rank': '8',
  'Title': '잘 지내자, 우리 (여름날 우리 X 로이킴)',
  'Artist': '로이킴로이킴',
  'SongID': '36595401'},
 {'Rank': '9', 'Title': '사랑은 늘 도망가', 'Artist': '임영웅임영웅', 'SongID': '34061322'},
 {'Rank': '10',
  'Title': '별 떨어진다 (I Do)',
  'Artist': '도경수(D.O.)도경수(D.O.)',
  'SongID': '36778869'},
 {'Rank': '11',
  'Title': '어떻게 이별까지 사랑하겠어, 널 사랑하는 거지',
  'Artist': 'AKMU (악뮤

## 차트 통합

In [9]:
import pandas as pd
import numpy as np

In [10]:
df = pd.read_csv(r"C:\Users\PRO\Desktop\datathon\melon_crawling_2017.csv")
df_1 = pd.read_csv(r"C:\Users\PRO\Desktop\datathon\melon_crawling_2024.csv")
df_2 = pd.read_csv(r"C:\Users\PRO\Desktop\datathon\melon_crawling_2025.csv")

df_2017_2024= pd.concat([df, df_1], ignore_index=True)
df_2017_2025= pd.concat([df_2017_2024, df_2], ignore_index=True)

df_2017_2025.head()

,Title,Artist
0,넘쳐흘러,엠씨더맥스 (M.C the MAX)엠씨더맥스 (M.C the MAX)
1,180도,벤벤
2,신용재,하은 (포맨)하은 (포맨)
3,너를 만나,폴킴폴킴
4,"모든 날, 모든 순간 (Every day, Every Moment)",폴킴폴킴


In [13]:
df_1

,Title,Artist
0,비의 랩소디,임재현임재현
1,헤어지자 말해요,박재정박재정
2,인사,범진범진
3,그대만 있다면 (여름날 우리 X 너드커넥션 (Nerd Connection)),너드커넥션 (Nerd Connection)너드커넥션 (Nerd Connection)
4,첫 눈,EXOEXO
...,...,...
35994,Dreaming,규현 (KYUHYUN)규현 (KYUHYUN)
35995,다시 사랑한다면,도원경도원경
35996,생각이나,부활부활
35997,손 닿으면,장범준장범준


In [11]:
df_2017_2025

,Title,Artist
0,넘쳐흘러,엠씨더맥스 (M.C the MAX)엠씨더맥스 (M.C the MAX)
1,180도,벤벤
2,신용재,하은 (포맨)하은 (포맨)
3,너를 만나,폴킴폴킴
4,"모든 날, 모든 순간 (Every day, Every Moment)",폴킴폴킴
...,...,...
58194,안녕 (Hello),조이 (JOY)조이 (JOY)
58195,On The Ground,로제 (ROSÉ)로제 (ROSÉ)
58196,가짜 아이돌,QWERQWER
58197,질풍가도 ('쾌걸 근육맨 2세' 1기 여는노래),유정석유정석


## 가수 두 번 적혀나왔으므로 절반

In [14]:
df_2017_2025['Artist'] = df_2017_2025['Artist'].apply(lambda x: x[:len(x)//2])

print(df_2017_2025.head())

                                   Title               Artist
0                                   넘쳐흘러  엠씨더맥스 (M.C the MAX)
1                                   180도                    벤
2                                    신용재              하은 (포맨)
3                                  너를 만나                   폴킴
4  모든 날, 모든 순간 (Every day, Every Moment)                   폴킴


In [15]:
df_2017_2025.to_csv("melon_2017_2025.csv", index=False, encoding="utf-8-sig")

## 중복 제거

In [17]:
df = pd.read_csv(r"C:\Users\PRO\Desktop\datathon\melon_2017_2025.csv")

df_unique = df.drop_duplicates(subset=["Title", "Artist"], keep="first")

df_unique.to_csv("melon_crawling_unique.csv", index=False)

## 가사 크롤링

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"C:\Users\PRO\Desktop\datathon\melon_crawling_unique.csv")
ques = (df["Title"] + " " + df["Artist"]).tolist()

print(ques)
df

['넘쳐흘러 엠씨더맥스 (M.C the MAX)', '180도 벤', '신용재 하은 (포맨)', '너를 만나 폴킴', '모든 날, 모든 순간 (Every day, Every Moment) 폴킴', '내 생에 아름다운 케이윌', '초록빛 폴킴', '너도 그냥 날 놓아주면 돼 윤건', '가을 타나 봐 바이브', '흔한 이별 허각', '고백 양다일', '이별하러 가는 길 임한별', '아름답고도 아프구나 비투비', '하루도 그대를 사랑하지 않은 적이 없었다 임창정', '열애중 벤', '신청곡 (Feat. SUGA of BTS) 이소라', '사계 (하루살이) 엠씨더맥스 (M.C the MAX)', '지나오다 닐로(Nilo)', '알았다면 장덕철', '너는 어땠을까 노을', '우리 그만하자 로이킴', '밤편지 아이유', '가을 안부 먼데이 키즈 (Monday Kiz)', '너 없인 안 된다 비투비', '동화 (feat. 아이유) 김동률', '그리워하다 비투비', '그때 헤어지면 돼 로이킴', '이별길 (GOODBYE ROAD) iKON', '멋지게 인사하는 법 (Feat. 슬기 of Red Velvet) Zion.T', '꿈처럼 내린 다비치', '그때가 좋았어 케이시 (Kassy)', 'Wait EXO', '길 (Song by 아이유, 헨리, 조현아, 양다일) god', 'Universe EXO', 'Beautiful Wanna One (워너원)', '사랑 알 수 없나봐 폴킴', '여기 있을게 (Smile On My Face) EXO', '첫눈처럼 너에게 가겠다 에일리(Ailee)', '그날처럼 장덕철', '선물 멜로망스', '시간을 견디면 엠씨더맥스 (M.C the MAX)', '사랑은 다 이러니 홍진영', '겨울이라 이런 거면 좋겠다 먼데이 키즈 (Monday Kiz)', 'Goodbye 웬디 (WENDY)', 'My Way 이수 (엠씨더맥스)', '동화 멜로망스', '취하고 싶다 (Prod. By 전태익) 황인욱', 'You 멜로망스', '주지마 로꼬, 화사 (HW

,Title,Artist
0,넘쳐흘러,엠씨더맥스 (M.C the MAX)
1,180도,벤
2,신용재,하은 (포맨)
3,너를 만나,폴킴
4,"모든 날, 모든 순간 (Every day, Every Moment)",폴킴
...,...,...
6524,시작의 아이,마크툽 (MAKTUB)
6525,사랑 노래,부석순 (SEVENTEEN)
6526,Doctor! Doctor!,ZEROBASEONE (제로베이스원)
6527,끊었어? (demo),"Chan (찬), 기원 (Giwon)"


In [5]:
import random
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from selenium.webdriver.chrome.service import Service


# User-Agent 리스트
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"
]

# 크롬 옵션 설정 함수
def get_chrome_options():
    options = webdriver.ChromeOptions()
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--headless')  # GUI 비활성화
    options.add_argument(f'user-agent={random.choice(user_agents)}')
    # options.add_argument(f'--proxy-server={random.choice(proxies)}')  # 프록시 설정
    options.add_argument('accept-language=ko-KR,ko;q=0.9')
    options.add_argument('accept-encoding=gzip, deflate, br')
    options.add_argument('dnt=1')  # Do Not Track 헤더 추가
    return options

# WebDriver 설정
path = 'c:/Users/PRO/.wdm/drivers/chromedriver/win64/131.0.6778.204/chromedriver-win32/chromedriver.exe'
service = Service(path)

url = 'https://www.melon.com/index.htm'
song_data = []

# 크롤링 시작
for i in range(0,len(ques)):
    options = get_chrome_options()
    driver = webdriver.Chrome(service=service, options=options)

    driver.get(url)

    try:
        # 검색 입력창 찾기 및 검색어 입력
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="top_search"]'))
        )
        search_box.clear()
        search_box.send_keys(ques[i])

        # 검색 버튼 클릭
        search_button = driver.find_element(By.XPATH, '//*[@id="gnb"]/fieldset/button[2]/span')
        search_button.click()

        # 첫 번째 검색 결과 클릭
        try:
            target_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="frm_songList"]/div/table/tbody/tr[1]/td[3]/div/div/a[1]/span'))
            )
            target_button.click()

            # 장르 값 가져오기
            genre = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[3]'))
            ).text

            # 가사 값 가져오기
            try:
                lyric = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="d_video_summary"]'))
                ).text.replace("\n", " ")
            except:
                lyric = "없음"

            print(f"'{ques[i]}' 성공.")
        except:
            genre, lyric = "없음", "없음"
            print(f"'{ques[i]}'에 대한 검색 결과가 없습니다.")
    except:
        genre, lyric =  "없음", "없음"
        print(f"'{ques[i]}' 검색 실패.")

    song_data.append({
        "Genre": genre,
        "Lyric": lyric
    })

    driver.quit()

    time.sleep(random.uniform(1, 10))

df = pd.DataFrame(song_data)
df.to_csv('melon_crawling.csv', index=False, encoding='utf-8-sig')
print(df.head())


'넘쳐흘러 엠씨더맥스 (M.C the MAX)' 성공.
'180도 벤' 성공.
'신용재 하은 (포맨)' 성공.
'너를 만나 폴킴' 성공.
'모든 날, 모든 순간 (Every day, Every Moment) 폴킴' 성공.
'내 생에 아름다운 케이윌' 성공.
'초록빛 폴킴' 성공.
'너도 그냥 날 놓아주면 돼 윤건' 성공.
'가을 타나 봐 바이브' 성공.
'흔한 이별 허각' 성공.
'고백 양다일' 성공.
'이별하러 가는 길 임한별' 성공.
'아름답고도 아프구나 비투비' 성공.
'하루도 그대를 사랑하지 않은 적이 없었다 임창정' 성공.
'열애중 벤' 성공.
'신청곡 (Feat. SUGA of BTS) 이소라' 성공.
'사계 (하루살이) 엠씨더맥스 (M.C the MAX)' 성공.
'지나오다 닐로(Nilo)' 성공.
'알았다면 장덕철' 성공.
'너는 어땠을까 노을' 성공.
'우리 그만하자 로이킴' 성공.
'밤편지 아이유' 성공.
'가을 안부 먼데이 키즈 (Monday Kiz)' 성공.
'너 없인 안 된다 비투비' 성공.
'동화 (feat. 아이유) 김동률' 성공.
'그리워하다 비투비' 성공.
'그때 헤어지면 돼 로이킴' 성공.
'이별길 (GOODBYE ROAD) iKON' 성공.
'멋지게 인사하는 법 (Feat. 슬기 of Red Velvet) Zion.T' 성공.
'꿈처럼 내린 다비치' 성공.
'그때가 좋았어 케이시 (Kassy)' 성공.
'Wait EXO' 성공.
'길 (Song by 아이유, 헨리, 조현아, 양다일) god' 성공.
'Universe EXO' 성공.
'Beautiful Wanna One (워너원)' 성공.
'사랑 알 수 없나봐 폴킴' 성공.
'여기 있을게 (Smile On My Face) EXO' 성공.
'첫눈처럼 너에게 가겠다 에일리(Ailee)' 성공.
'그날처럼 장덕철' 성공.
'선물 멜로망스' 성공.
'시간을 견디면 엠씨더맥스 (M.C the MAX)' 성공.
'사랑은 다 이러니 홍진영' 성공.
'겨울이라 이런 거면 좋겠다 먼데이 키즈 

: 